In [1]:
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
tf.set_random_seed(0)

In [2]:
mnist = read_data_sets("..\\Data\\MNIST_Data", one_hot=True, reshape=False, validation_size=0)

Extracting ..\Data\MNIST_Data\train-images-idx3-ubyte.gz
Extracting ..\Data\MNIST_Data\train-labels-idx1-ubyte.gz
Extracting ..\Data\MNIST_Data\t10k-images-idx3-ubyte.gz
Extracting ..\Data\MNIST_Data\t10k-labels-idx1-ubyte.gz


In [3]:
# 28 * 28 = 784
x = tf.placeholder(tf.float32, [None, 28, 28, 1], name="InputData")
y = tf.placeholder(tf.float32, [None, 10], name="Output")

In [4]:
W = tf.Variable(tf.zeros([784, 10]), name="Weights")
b = tf.Variable(tf.zeros([10]), name="Biases")

In [5]:
with tf.name_scope("Wx_b") as scope:
    # flatten the images into a single line of pixels
    # -1 in the shape definition means "the only possible dimension that will preserve the number of elements"
    __X = tf.reshape(x, [-1, 784])
    model = tf.nn.softmax(tf.matmul(__X, W) + b)

In [6]:
W_h = tf.summary.histogram("weights", W)
b_h = tf.summary.histogram("biases", b)

In [7]:
with tf.name_scope("cost_func") as scope:
    cost_function = -tf.reduce_sum(y * tf.log(model))
    tf.summary.scalar("cost_func", cost_function)

In [8]:
learning_rate = 0.01
training_iteration = 30
batch_size = 100
display_step = 2

In [9]:
with tf.name_scope("train") as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [10]:
init = tf.global_variables_initializer()

In [11]:
merged_summary_op = tf.summary.merge_all()

In [12]:
with tf.Session() as sess:
    sess.run(init)
    summary_writer = tf.summary.FileWriter('Logs', graph=sess.graph)
    
    for iteration in range(training_iteration):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x:batch_xs, y:batch_ys})
            avg_cost += sess.run(cost_function, feed_dict={x:batch_xs, y:batch_ys}) / total_batch
            
            summary_str = sess.run(merged_summary_op, feed_dict={x:batch_xs, y:batch_ys})
            summary_writer.add_summary(summary_str, iteration*total_batch + i)
            
        if iteration % display_step == 0:
            print("Iteration : " + str(iteration + 1) + " Cost = " + str(avg_cost))
            
    print("completed")
    predication = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(predication, tf.float32))
    print("Accuracy :" + str(accuracy.eval({x: mnist.test.images, y:mnist.test.labels})))

Iteration : 1 Cost = 29.7747808389
Iteration : 3 Cost = 20.8320865202
Iteration : 5 Cost = 20.0496964359
Iteration : 7 Cost = 19.5401901547
Iteration : 9 Cost = 19.215731376
Iteration : 11 Cost = 19.0707324656
Iteration : 13 Cost = 18.8595597132
Iteration : 15 Cost = 18.7534988252
Iteration : 17 Cost = 18.5952136771
Iteration : 19 Cost = 18.4894305333
Iteration : 21 Cost = 18.5211666187
Iteration : 23 Cost = 18.438094033
Iteration : 25 Cost = 18.2633220132
Iteration : 27 Cost = 18.0477957066
Iteration : 29 Cost = 18.0980408327
completed
Accuracy :0.9255
